# Generating time series of surface mass balance

Now that we have aggregated surface mass balance to catchments, explored the data, and identified some good candidate temporal models, we are ready to generate time series of SMB.  We will apply a selected temporal model and a spatially-informed noise method to generate a range of SMB time series with variability that matches that presented in process models.

UPDATE 27-28 DEC 22: Read in catchment mean series rather than totals, and include all basins (range(0,260)) rather than only 1-200.

### Preliminaries

Make sure that you have activated your `stisp` environment, defined for conda installation in the `environment.yml` file of this repository.  All of the packages we import here should be available within that environment.

In [ ]:
from sklearn.covariance import GraphicalLassoCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from statsmodels.tsa.ar_model import AutoReg
import scipy.linalg
import glob

Adjust matplotlib default settings for plot label sizes and color scheme:

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

noise_colors=cm.get_cmap('Blues')

We do all of our stochastic fitting to the output from individual process models, because fitting a multi-model mean would damp out some of the variability that we want to capture.  Data is aggregated to catchments defined by Mouginot & Rignot (link), tagged with ID numbers consistent with that dataset.  Here we select a process model to emulate and a glacier catchment to examine.

In [ ]:
model_names = ['ANICE-ITM_Berends', 'CESM_kampenhout', 'dEBM_krebs','HIRHAM_mottram', 
                'NHM-SMAP_niwano', 'RACMO_noel', 'SNOWMODEL_liston']

highlight_model = 'ANICE-ITM_Berends'
highlight_catchment_name, highlight_catchment_id = 'KANGERLUSSUAQ', 101

Now, we define some helper functions.  `read_catchment_series` will read in our aggregated data from CSV, `fit_catchment_series` will test several orders of AR(n) model and report which is the best fit to the data, and `find_AR_residuals` returns the residuals from a selected AR(n) model fit, which we will need for the spatial covariance method.

In [ ]:
## Read in time series
def read_catchment_series(fpath, anomaly=True):
    catchment_fpath = fpath
    catchment_tseries = pd.read_csv(catchment_fpath, index_col=0, parse_dates=[0])
    catchment_tseries.mask(catchment_tseries>1e30)
    anomaly_series = catchment_tseries - catchment_tseries.mean()
    if anomaly:
        return anomaly_series
    else:
        return catchment_tseries

def fit_catchment_series(tseries, which_model, comparison_n=range(1,6), 
                         seasonal=True):
    bic_per_n = pd.DataFrame(index=comparison_n, columns=model_names)
    
    if 'multi' in which_model:  ## allow multi-model mode reporting
        for m in model_names:
            for n in comparison_n:
                mod = AutoReg(tseries[m], n, trend='ct', seasonal=seasonal)
                results = mod.fit()
                bic_per_n[m][n] = results.bic
            bic_per_n[m] = pd.to_numeric(bic_per_n[m])
        best_n = bic_per_n.idxmin().mode()[0]
    else:
        for n in comparison_n:
            mod = AutoReg(tseries[which_model], n, trend='ct', seasonal=seasonal)
            results = mod.fit()
            bic_per_n[which_model][n] = results.bic
        bic_per_n[which_model] = pd.to_numeric(bic_per_n[which_model])
        best_n = bic_per_n[which_model].idxmin()
    
    bic_difference = bic_per_n.transform(lambda x: x-x.min())
    
    return best_n, bic_difference

def find_AR_residuals(tseries, which_model, chosen_n=1, 
                         seasonal=False):
    mod = AutoReg(tseries[which_model], chosen_n, trend='ct', seasonal=seasonal)
    results = mod.fit()
    resids = results.resid
    
    return resids

## Time series from AR(n) fits

First, we read in process-model-derived time series of SMB for the selected catchment, from all process models.  We resample the monthly data to annual sums with `pandas.Series.resample('A').sum()`.  

We identify the best autoregressive model according to the most common best-fit among all process models, so that we can enforce some consistency later on.  You can easily change this by moving the fit into the `for` loop and setting `which_model=m` rather than `multi`.

Finally, we fit an autoregressive model to each process model's output using `statsmodels.tsa.ar_model.AutoReg().fit()`, and we save the fitted AR models to a dictionary indexed by process-model name.

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

We ignore warnings for now, because statsmodels is going to complain about parameter names.  This does not affect our results.  If you are debugging and want to see warnings, you can set `action=once` instead.

In [ ]:
## Time series from AR(n) fits
mod_fits = {m: [] for m in model_names}
mods = {m: [] for m in model_names}

ctmt_fpath = glob.glob('/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP-processed/*-catchment_{}_mean-tseries.csv'.format(highlight_catchment_id))[0]
s = read_catchment_series(ctmt_fpath, anomaly=True)
a = s.resample('A').sum()
best_n, _ = fit_catchment_series(a, which_model='multi', seasonal=False)
for m in model_names:
    mod = AutoReg(a[m], best_n, trend='ct', seasonal=False).fit()
    fv = mod.fittedvalues
    r = mod.resid
    mod_fits[m] = fv
    mods[m] = mod

Now we have our dictionary `mods` that stores AR models trained on process model output.  The elements of `mods` are `AutoRegResults` objects.  We can use the built-in method `predict` to generate several realizations of stochastic surface mass balance that approximate a given process model's series.

In [ ]:
## 12 Mar 2023
## What happens if we insist on AR(1) for this example?
best_n = 4
for m in model_names:
    mod = AutoReg(a[m], best_n, trend='ct', seasonal=False).fit()
    fv = mod.fittedvalues
    r = mod.resid
    mod_fits[m] = fv
    mods[m] = mod

    ## Answer: the noise does an okay job, but the range in the projections is about 2x as large

In [ ]:
n_realizations = 10
preds = []
for k in range(n_realizations):
    mod = mods[highlight_model]
    ar_smb_k = mod.predict(best_n, len(a)-1, dynamic=True)
    preds.append(ar_smb_k)

Note that we ask `predict` to use dynamic prediction (not using the input data values) after a short initial period.  You can experiment with different lengths of the initial period, but if you do not set the `dynamic` argument, the default behavior is that `predict` uses all available input values.  See the [statsmodels documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.ar_model.AutoRegResults.predict.html#statsmodels.tsa.ar_model.AutoRegResults.predict) for more details.

Let's plot the AR(n) realizations in `preds` and compare them with the SMB process model output.

In [ ]:
plt.clf()
fig1, ax1 = plt.subplots(figsize=(10,6))
for k in range(n_realizations):
    ax1.plot(preds[k], 
            color=noise_colors(k/n_realizations), alpha=0.5)
ax1.plot(a[highlight_model], color='k',
         label='{} output'.format(highlight_model))
ax1.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='AR({}) realizations'.format(best_n))
ax1.set(xlabel='Year', ylabel=r'Catchment SMB anomaly [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax1.legend(loc='best')
plt.show()

In [ ]:
## TODO: figure out generation of distinct AR(n) realizations...these should not all coincide

## Noise with spatial information

Now, we will find a sparse correlation matrix for _all_ catchments, fit with the same order of AR model, to a single process model's SMB output.

First we find and store the residuals of the AR fit to each catchment.  Then we find the empirical correlation matrix `emp_C`.  

In [ ]:
ar_resids = []
ts_toplot = []
for i in range(0, 260):
    # print(i)
    ctmt_fpath = glob.glob('/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP-processed/*-catchment_{}_mean-tseries.csv'.format(i))[0]
    s = read_catchment_series(ctmt_fpath, anomaly=True)
    a = s.resample('A').sum()
    ts_toplot.append(a)
    r = find_AR_residuals(a, which_model=highlight_model, chosen_n=best_n, seasonal=False)
    ar_resids.append(r)

ar_resids -= np.mean(ar_resids, axis=0) # normalize
emp_C = np.corrcoef(ar_resids)

For our case, where we have 30 years of data for 260 catchments, the empirical correlation matrix is unstable and contains too much information.  We use `sklearn.GraphicalLassoCV()` to find a sparse correlation matrix that approximates it and tamps down some of the unwanted features.

In [ ]:
np.random.seed(0)
X = np.random.multivariate_normal(mean=np.zeros(len(ar_resids)), cov=emp_C, size=len(ar_resids[0]))

gl_model = GraphicalLassoCV()
gl_model.fit(X)
cov_ = gl_model.covariance_

Now, following Hu \& Castruccio (2021 preprint), we generate noise series for each catchment using the sparse covariance matrix (in Cholesky lower-triangular decomposition) and the diagonal matrix of standard deviations of the individual catchments' AR residuals.  Here, the noise series will be the same length as `ar_resids`, representing however many years are in the training set.

In [ ]:
L = scipy.linalg.cholesky(cov_, lower=True)
N = np.random.normal(size=np.shape(ar_resids)) # draws from normal dist

D = np.diag(np.std(ar_resids,1)) ## diagonal matrix of standard devs
# scaled_noise = D @ L @ N

noise_realizations = []
for j in range(n_realizations):
    Nj = np.random.normal(size=np.shape(ar_resids))
    noise_j = D @ L @ Nj
    noise_realizations.append(noise_j[highlight_catchment_id-1])

Plot the noise realizations alone:

In [ ]:
## Plot noise and process model residual
fig2, ax2 = plt.subplots(figsize=(10,6))
for k in range(n_realizations):
    ax2.plot(a.index[best_n::], noise_realizations[k], 
            color=noise_colors(k/n_realizations), alpha=0.5)
ax2.plot(ts_toplot[highlight_catchment_id][highlight_model], color='k', ## previously used highlight_catchment_id-1 as index, but now we also read in catchment 0, so the -1 is not needed.
         label='{} output'.format(highlight_model))
ax2.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax2.set(xlabel='Year', ylabel=r'Residual SMB anomaly [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax2.legend(loc='best')
plt.show()

## A full stochastic model of SMB

We can put together the AR(n) series and the spatially informed noise to generate "full" realizations of SMB.  For example:

In [ ]:
## Plot a sum of the two
fig3, ax3 = plt.subplots(figsize=(10,6))
for k in range(n_realizations):
    ax3.plot(mods[highlight_model].predict(best_n, len(a)-1, dynamic=best_n)
             +noise_realizations[k], 
            color=noise_colors(k/n_realizations), alpha=0.5)
ax3.plot(ts_toplot[highlight_catchment_id][highlight_model], color='k',
         label='{} output'.format(highlight_model))
ax3.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax3.set(xlabel='Year', ylabel=r'Catchment SMB anomaly [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax3.legend(loc='best')
plt.show()

We can even forecast SMB into the future with this combination.  Simply choose how many years of output should be created, feed this information to `predict`, and perform another matrix multiplication to generate noise series of the appropriate length.

In [ ]:
## Future forecast
yrs_after_1980 = 70
noise_into_future = []
preds_into_future = []
for j in range(n_realizations):
    ar_smb_k = mods[highlight_model].predict(best_n, yrs_after_1980, dynamic=True)
    preds_into_future.append(ar_smb_k)
    Nj = np.random.normal(size=(len(ar_resids), yrs_after_1980))
    noise_j = D @ L @ Nj
    noise_into_future.append(noise_j[highlight_catchment_id-1])

fig4, ax4 = plt.subplots(figsize=(10,6))
for k in range(n_realizations):
    ax4.plot(preds_into_future[k]
             +noise_into_future[k][best_n-1::], 
            color=noise_colors(k/n_realizations), lw=1.5, 
             alpha=0.8)
ax4.plot(ts_toplot[highlight_catchment_id][highlight_model], 
         color='k', lw=2.0,
         label='{} output'.format(highlight_model))
ax4.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax4.set(xlabel='Year', ylabel=r'Catchment SMB anomaly [mm w.e. a$^{-1}$]')
# ax4.set(xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
#                 np.datetime64('2000-01-01'), np.datetime64('2010-01-01'),
#                 np.datetime64('2020-01-01'), np.datetime64('2030-01-01'),
#                 np.datetime64('2040-01-01'), np.datetime64('2050-01-01'),),
#         xticklabels=(1980,1990,2000,2010,2020,2030,2040,2050))
ax4.legend(loc='best')
plt.show()